# AtLAST Telescope Simulator - Line Emission 

One of the key first light instruments for AtLAST will be some kind of high spectral resolution camera, operating at at least one waveband at a time.  This Jupyter notebook creates a simplistic simulation of what we can expect the performance of that telescope/instrument combination to be using the AtLAST Sensitivity Calculator as its starting point.

This notebook is provided as an example of how to setup a telescope simulation for AtLAST using the Sensitivity Calculator.  You can either modify it to suit your own use case / input file, or create your own using this as a guide.  

## What this Simulator *can* do:
* Take observing constraints in the same way the sensitivity calculator does
* Import a user suplied FITS file to simulate observing
* Convolve the FITS file to the resolution expected at the requested observing frequency
* Add a constant gaussian noise level to the image consistent with what AtLAST would expect to obtain (either as specified, or given the amount of integration time requested)


## What this Simulator *can't* do:
* Simulate any spacings between individual detectors/pixels (out of scope with no instrument specification)
* Simulate fall off in sensitivity as a function of distance from pointing centre / chopping constraints (out of scope with no detailed calibration strategy analysis)


## Stepping through this example Notebook

This example notebook is provided as a reference/guide on how to setup an AtLAST simulation given an input FITS file.  The steps below include:

1. Setup the simulator (importing libraries, setting up plotting routines, etc)
2. Input the user specified quantities (time/sensitivity request, observing parameters, input FITS file, etc)
3. Convolve the input file to the (spatial and spectral) resolution expected for an AtLAST observation at the given input frequency
4. Compare the input image to the expected instrument footprint (using number of pixels)
4. Determine the noise level to apply to the data
5. Apply a gaussian noise level to the data
6. Show a collapsed image and an averaged spectrum
7. Inform the user how many instrument footprints are required to image their FoV.





# Setup The Simulator

Below we import all of the relevant python packages for the sensitivity calculation and telescope simulation, including setting up for importing and displaying FITS images

There are also cells which setup functions for use later in the notebook.

## Import the AtLAST Sensitivity Calculator package

In [ ]:
from atlast_sc.calculator import Calculator

## Import astronomy specific packages

In [ ]:
# to use astropy coordinates and units
import astropy.units as u  # for ensuring units are treated properly
import astropy.constants as const # to make use of the astropy constants package

# to handle FITS files
from astropy.io import fits # to import and manipulate FITS files
from astropy.wcs import WCS # to use WCS information from the FITS headers

# to reproject the convolved image onto a new grid
from reproject import reproject_interp

## Visualisation Imports

In [ ]:
#Import Matplotlibs pyplot package

import matplotlib.pyplot as plt

# Import the Rectangle patch to draw the FoV on the final image
from matplotlib.patches import Rectangle

# show the plots inline
%matplotlib inline

# to allow for wide figures as required
plt.rcParams['figure.figsize'] = [18, 8]

# suppress the warnings output by astropy when headers are accessed
import warnings
warnings.filterwarnings('ignore')

## Imports for Gaussian noise

In [ ]:
# gaussian noise imports
import numpy as np
import math

## User defined functions used in the rest of this notebook

Here we define all of the functions to be used later. This includes a downsampling algorithm which calculates what the refactored header should have as its keywords, and a function which converts between velocity and frequency (at a given frequency) to help with bandwidth calculations.  Note that the conversion being done here is different from the `astropy` equivalencies package which converts the difference between input value and rest frequency into the equivalent units, rather than expecting a bandwidth to be converted. The equation used here is:

$$ \frac{\Delta \nu}{\nu} = \frac{\Delta v}{c}$$

In [ ]:
# downsampling function that changes the relevant keywords in the user provided header by scaling them down by a factor of `factor`

def downsample(header,calculator,sampling):
    inhead = header
    modhead = header.copy()
    
    # get current resolution, and derive scaling to reqired
    dish_diam = 2* calculator.instrument_setup.dish_radius.value
    
    # calculate spatial CDELT value for new header
    spat_cd = (1.2 * (const.c / calculator.obs_freq) / dish_diam )/sampling
    spat_fact = (inhead['CDELT1']/spat_cd.decompose()).value # ratio between old and new spatial resolution
    
    # calculate spectral CDELT value for new header
    spec_cd = calculator.bandwidth.to('Hz').value
    spec_fact = inhead['CDELT3']/spec_cd

    # loop over CDELT and scaling factors to set header variables
    scale = np.abs([spat_fact,spat_fact,spec_fact])
    print(scale)
    
    for i in [1,2,3]:  #(x,y,z) scaling
        modhead[f'CDELT{i}'] = inhead[f'CDELT{i}']*scale[i-1]
        modhead[f'CRPIX{i}'] = inhead[f'CRPIX{i}']/scale[i-1]
        modhead[f'NAXIS{i}'] = int(inhead[f'NAXIS{i}']/scale[i-1])
    
    
    
        
    return modhead



    



# convert between velocity and frequency for bandwidth calculations
def equiv(obs_freq, in_bandwidth):
    if in_bandwidth.unit.is_equivalent(u.km/u.s):  # calculate bandwidth in frequency
        output = (in_bandwidth * obs_freq / const.c).to(u.MHz)
    elif in_bandwidth.unit.is_equivalent(u.Hz):  # calculate bandwidth in velocity
        output = (in_bandwidth * const.c / obs_freq).to(u.km/u.s)
    else:
        print('units not consistent with velocity or frequency, returning original')
        output = in_bandwidth
    return output
        

# Input User Specified Quantities

Here we generate an instance of the sensitivity calculator which we use for the rest of this simulation notebook.  The first part of this section shows you what the default settings are for the calculator to:
1) Show what the default settings are
2) Show the naming convention for the user editable parameters

We then go through changing some of those parameters and displaying how that changed the variables used for the sensitivity or integration time calculation, followed  by setting the number of pixels to be simulated in this notebook, and the input FITS image to be used in the simulation.  The simulator assumes a fully sampled array of pixels arranged in a square because we don't know the exact properties of any of the instruments expected for AtLAST.  

## Initialise calculator and see default parameters

In [ ]:
# create an instance of the calculator
calculator = Calculator()
#show the default values used in the calculation
print(calculator.user_input)

Listed above are the parameters most commonly edited in the sensitivity calculator. Note that both integration time and sensitivity are listed here, while the calculator only uses one of them at a time to derive the other.  In addition to `calculator.user_input`, there are two other files that could be output here:

`calculator.instrument_setup`: Gives a number of changable parmeters such as the telescope diameter - for those looking to see the effects of having a smaller diameter

`calculator.derived_parameters`: Using the input parameters, there are a number of parameters the simulator derives. Users can inspect those values by printing this out.


## Change some of the input parameters

Now that we know what the variable names are for the input parameters, we can change some of them to reflect the simulated observation we want to make. Specifically lets change the spectral resolution to be the equivalent of 1 km/s. For this, we need to use the `astropy` equivalencies package.

In [ ]:
# set an integration time
calculator.t_int = 3000 *u.s

# change the observing frequency to see how the user input changes
calculator.obs_freq = 345.*u.GHz

# change the observing bandwidth by specifying a velocity resolution (in km/s)
velo = 1 *u.km/u.s
bandwidth = equiv(calculator.obs_freq,velo)  # use the function above to convert to frequency
calculator.bandwidth = bandwidth

# print out the input parameters to show that they've changed
print(calculator.user_input)

## Specify the image to be used in the simulation

Ideally, here you would specify an oversampled base image to which the simulator can attach a gaussian noise level derived from the sensitivity calculator. In this example, we use an ALMA archive spectral cube of the Orion Bar because it is chemically rich, and at better (spatial) resolution than we expect to get with AtLAST. The input FITS file can be downloaded from the ALMA archive, and is not included in this directory.



In [ ]:
#import an example image
filename = 'member.uid___A002_X6dddc4_X6a.Orion_Bar.COnocont.Clean.image.fits'  
hdu = fits.open(filename)[0]
wcs = WCS(hdu.header)

#ALMA and reproject use different cases for UTC (uppercase vs. lowercase)
# convert the ALMA header to meet the reproject requirement
hdu.header['TIMESYS'] = 'utc'

## Set the number of pixels to be simulated

Here we setup the number of pixels expected for the instrument. For a spectral line reciever, we hope to achieve of order 1000 pixels, while for a continuum camera, we expect much closer to $10^6$.  Because this is a line example, we'll use 1000 as the number of pixels here.

As noted above, we cannot take pixel spacing into account because we don't have specifications for our cameras. As such, this simulator simply places a square box containing as close to the input number of pixels as possible (scaling up slightly to provide an integer number of pixels in either spatial direction).

In [ ]:
# simulator/imaging settings
Npix_detector = 1000
len_detector = math.ceil(np.sqrt(Npix_detector))

## Calculate pixel scale / beam size based on observing frequency and telescope diameter

In [ ]:
# pull the telescope radius from the instrument_setup
radius = calculator.instrument_setup.dish_radius.value
# pull the observing frequency from the user inputs
freq = calculator.user_input.obs_freq.value

theta = ((1.2* const.c / (freq * (2*radius) ))*u.radian).to('arcsec')
print(f'Given the requested observing frequency, the resolution of AtLAST will be {theta:0.2f}')

## Set the pixel sampling of the convolved image

When the image is convolved to the resolution of AtLAST, we need to downsample the pixel scaling accordingly. Usual rule of thumb is 3-5 pixels per resolution element. That sampling is set here for use in section 4

In [ ]:
sampling = 5  # provide 5 pixels for each resolution element of the instrument

# Display the input data and convolve it to the resolution of AtLAST

Now that we have input data, and the sensitivity calculator setup, we can begin the process of simulating an observation with AtLAST.  The first step in that process is to convolve the input data to the resolution expected for AtLAST.  Below we:

* Show an averaged (collapsed) image cube at its original spatial resolution and a spectrum averaged over the imaged area
* Derive the smoothing kernel required to convolve that data to the spatial and spectral resolution of AtLAST 
* Resample the input data to the AtLAST resolution
* Show a small area of the input image at original and convolved resolution to show what happened
* Compare the size of the input image to the expected instrument footprint (instrument field of view)


## Show the original data

In [ ]:
# read the data from the input FITS file
data = hdu.data
# give data units from header (need capitalize in order to give Jy instead of JY, which astropy.units complains about)
data=  data*u.Unit(hdu.header['bunit'].capitalize()) 

# set the data plotting limits by finding the min/max values over most of the image (removing edges), and scaling the max to 10% of max to avoid spikes and saturated pixels
vmin,vmax = np.nanmin(data[0,:,10:-10,10:-10]).value,np.nanmax(data[0,:,10:-10,10:-10]).value
noise = np.nanstd(data)

### show collapsed image

#collapse over the (degenerate) stokes and spectral axes
img_collapse = np.nanmean(data,axis=(0,1))

# show the input data
fig = plt.figure()
ax = fig.add_subplot(projection=wcs.celestial)  # add the WCS information to the plot
im = ax.imshow(img_collapse.value,origin='lower',cmap='twilight_r')
# add a colorbar
cbar = plt.colorbar(im)
cbar.set_label(data.unit)

### show collapsed spectrum

# collapse over the (degenerate) stokes and spatial axes
spec_collapse = np.nanmean(data,axis=(0,2,3))
# get values of spectral axis from the header
spec_axis = wcs.spectral.array_index_to_world_values(list(range(0,len(spec_collapse))))

# show the input data
fig = plt.figure()
ax = fig.add_subplot()
spec = ax.plot(spec_axis,spec_collapse)
    


## Resample the data to the (requested) resolution of AtLAST

Use the header information of the original image to find its pixel scale, and calculate the resampling factors required to create an AtLAST observation at the user specified spectral resolution and spatial resolution set by the telescope diameter

In [ ]:
# convolve the image to the beamsize at the requested observing frequency
#pix_scale = (hdu.header['cdelt1']*u.deg).to('arcsec')
#kern = (theta/pix_scale).value
#kernel = Gaussian2DKernel(x_stddev=kern)

# convolve the data with the gaussian kernel, and save the results for the reprojection below.
#astropy_conv = convolve(data,kernel)



## Downsample to the AtLAST pixel scale

Convolving the input image above doesn't change the size of the pixels in the original image, only smooths it.  Here we use the `reproject` package to do that downsampling on the data, and use the downsamplig function to update the header keywords accordingly.

In [ ]:
#create a downsampled header using the 'downsample' function and sampling set in sections 2.5 and 3.6 (respectively)
ds_header = downsample(hdu.header,calculator,sampling)

# reproject the convolved image to the downsampled header
array,footprint = reproject_interp((hdu.data,hdu.header),ds_header)


## Resample to AtLAST resolution

Plot the original and the convolved/regridded data to show how the convolution has changed how the data look. 

In [ ]:
# read the resampled data and give units from header 
newdata=  array*u.Unit(hdu.header['bunit'].capitalize()) 
# load in the new wcs information
newwcs = WCS(ds_header)

### show collapsed images
# iterate over the original and resampled images
indata = [data,newdata]
inwcs = [wcs,newwcs]
labels = ['Original','Resampled']

fig = plt.figure()
for i in (0,1):

    #collapse over the (degenerate) stokes and spectral axes in the resampled data
    img_collapse = np.nanmean(indata[i],axis=(0,1))

    # show the resampled data
    ax = fig.add_subplot(1,2,i+1,projection=inwcs[i].celestial)  # add the WCS information to the plot
    im = ax.imshow(img_collapse.value,origin='lower',cmap='twilight_r')
    ax.set_title(labels[i])
    # add a colorbar
    cbar = plt.colorbar(im)
    cbar.set_label(data.unit)


### show collapsed spectra
# iterate over the original and resampled spectra

fig = plt.figure()
ax = fig.add_subplot()

for i in (0,1):
    # collapse over the (degenerate) stokes and spatial axes
    spec_collapse = np.nanmean(indata[i],axis=(0,2,3))
    # get values of spectral axis from the header
    spec_axis = inwcs[i].spectral.array_index_to_world_values(list(range(0,len(spec_collapse))))
    # show the spectrum and give it a label
    spec = ax.plot(spec_axis,spec_collapse,label=labels[i])

    # show the legend on the plot
ax.legend()
  

# Evaluate image size compared to observing footprint

As the number of pixels in a detector varies, so does the amount of the telescope field of view it can capture in a single setup. Here we look at whether the input image is large enough to fill the instrument footprint 

**Image smaller than footprint**

If the image is not big enough to fill the instrument footprint, then the image will be padded out with zeros to show how large the instrument footprint would be in comparison to the input image. In this case the convolved input image is centred in the new, zero padded, array, and the header is updated accordingly. Here, the instrument footprint is also plotted on the image.

**Image larger than footprint**

If the image is larger than the instrument footprint, then the code below draws a red box in the centre of the image to indicate the footprint, and also outputs how many observations (and therefore on-source integration time) are required to observe the entire image, rounding up the number of footprints to an integer value.



In [ ]:
array.shape

In [ ]:
# get the shape of the input (convolved) image
s1,z1,y1,x1 = array.shape  #badkwards here to make the rest make sense (see note above about python vs. FITS handling of first axis)

# find which is larger, the x or y coordinate, or the number of detector pixels across that dimension
pad = np.max((x1,len_detector)),np.max((y1,len_detector))


header = ds_header.copy()  #create a new copy of the header information

# determine if the image is smaller than the instrument footprint in either x or y

# yes, one or more dimensions are smaller
if x1 <= len_detector or y1 <= len_detector:
    for i in (0,1):

        cp = 'CRPIX{0:d}'.format(i+1)
        na = 'NAXIS{0:d}'.format(i+1)
        header[cp] = int(math.ceil(pad[i]/2))
        header[na] = int(pad[i])

    # find the padding required to create that array & add zeros to the data
    pads = [int((len_detector-x1)/2),int((len_detector-y1)/2)]
    pads.insert(1,pads[0])
    pads.insert(-1,pads[-1])
    pads = [0 if i < 0 else i for i in pads]
    
    # create the padded data array using the numpy 'pad' module
    array = np.pad(array,((pads[2:]),(pads[:2])),'constant',constant_values=0)

# no, the image is larger than the instrument footprint    
else:
    # because the image is larger than the footprint, just pass the original array along for plotting
    array = array    

fig = plt.figure()
ax = fig.add_subplot(projection=WCS(header).celestial)
img_collapse = np.nanmean(array,axis=(0,1))
ax.imshow(img_collapse,origin='lower',cmap='twilight_r')

# plot the detector footprint on the image (regardless of whether the image needed padding.
#find where the first (bottom left) pixel of the rectangle should be
startx = np.max((int(x1/2 - len_detector/2),0))
starty = np.max((int(y1/2 - len_detector/2),0))

# use the Rectangle patch to specify a box starting at (startx,starty) with dimensions of len_detector x len_detector
rect = Rectangle((startx,starty),len_detector,len_detector,edgecolor='r',facecolor='None',lw=5)
ax.add_patch(rect)




# Determine the noise level to add to the image

With the input image sufficiently processed, and the size of the instrument footprint compared to the image size, we can now setup use the sensitivity calculator to derive an integration time or sensitivity limit.

In the example below, we use an integration time to derive a sensitivity limit, and then apply that noise level to the image.

## Calculate sensitivity based on integration time

Using the integration time set in section 3.2, we can calculate the single pixel sensitivity using the sensitivity calculator.

In [ ]:
# copy the integration time from section 3.2
integration_time = calculator.t_int

# use telescope size to convert point source sensitivity to flux density 
# (assuming no beam dilution)
calculated_sensitivity = calculator.calculate_sensitivity(integration_time).to(u.mJy)/theta**2
print("Sensitivity: {:0.5f} for an integration time of {:0.2f} ".format(calculated_sensitivity, integration_time))

## Add Gaussian noise to the image based on serived sensitivity calculation.

Using the sensitivity above, create a noise map using the numpy gaussian (random normal) distribution module

In [ ]:
# sigma of the distribution comes from the sensitivity calculator
sigma = calculated_sensitivity.to('mJy/arcsec2').value
mean = 0 # trust that the continuum was subtracted properly

gaussian_noise = np.random.normal(mean, sigma, (array.shape))*u.MJy/u.sr

# add the noise to the image
noisy_image = array+gaussian_noise.value

# Plot the final results

Now, with the gaussian noise level applied to the convolved and resampled image, we have the final simulated AtLAST observation to plot.  

Below we plot 4 images, representative of the different steps followed through in this notebook:

* the original data
* the data convolved to the resolution of AtLAST
* the Gaussian noise map applied to the image
* the final image

For the final 3, a single instrument footprint has been added to the image, as above in section 5.

In [ ]:
inputs = [data.value,array,gaussian_noise.value,noisy_image]
inwcs = [wcs,newwcs,newwcs,newwcs]
titles = ['Input Data', 'Convolved to AtLAST Resolution',
          'Noise Level of Observation', 'Observed Image']

# calculate vmin/vmax for all images (from resampled input data)
vmin,vmax = np.nanmin(np.nanmean(array,axis=(0,1))),np.nanmax(np.nanmean(array,axis=(0,1)))
fig = plt.figure()
for i in range(0,len(inputs)):

    ax = fig.add_subplot(2,2,i+1, projection=wcs.celestial)
    img_collapse = np.nanmean(inputs[i],axis=(0,1))
    im=ax.imshow(img_collapse,vmin=vmin,vmax=vmax,origin='lower',cmap='twilight_r')
    if i > 0:  # don't plot the instrument footprint on the original image (the way its pixels are counted is different)
        rect = Rectangle((startx,starty),len_detector,len_detector,edgecolor='r',facecolor='None',lw=5)
        ax.add_patch(rect)
    ax.set_title(titles[i])
    cbar = plt.colorbar(im)
    cbar.set_label(data.unit)
plt.tight_layout()   
    
fig = plt.figure()
ax = fig.add_subplot()
for i in range(0,len(inputs)):
    # collapse over the (degenerate) stokes and spatial axes
    spec_collapse = np.nanmean(inputs[i],axis=(0,2,3))
    # get values of spectral axis from the header
    spec_axis = inwcs[i].spectral.array_index_to_world_values(list(range(0,len(spec_collapse))))
    # show the spectrum and give it a label
    spec = ax.plot(spec_axis,spec_collapse,label=titles[i])

    # show the legend on the plot
ax.legend()

## Mapping Constraints

Here, we evaluate whether mapping is required, and if so, the (integer) number of instrument footprints required to cover the input image.  The integration times reported here are a lower limit, and do not include any of the following:

* array configurations that aren't fully sampled
* array configurations that aren't square
* off source (calibration time)

In [ ]:
#find how many pointings of an N pixel detector would be required to observe the full image

# number of pixels in the final image
s,z,a,b = noisy_image.shape
Npix_img = a*b
N_int_actual = Npix_img/Npix_detector
# find the whole number of footprints required to map the whole image
N_int_ceiling = math.ceil(N_int_actual)

# based on the number of footprints required, create the appropriate text to append to the overall output print statement.
if N_int_ceiling > 1:
    footprint = f'To fully image your map will require {N_int_ceiling} integrations, and a total on source time of {integration_time*N_int_ceiling}'

elif N_int_ceiling<= 1:
    footprint = f'The total on source integration time is {integration_time} (assuming the detector is fully sampled).'

# tell the user how long their observation will take
print(f'There are {Npix_img} pixels in the image, and {Npix_detector} on the detector, \
so {N_int_actual} integrations are required.  {footprint}')
